<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2004/04_Tratamiento_de_valores_at%C3%ADpicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paquetes

In [ ]:
pip install pandas==1.5.3 &> /dev/null

In [ ]:
pip install outlier_utils  &> /dev/null

In [ ]:
pip install outliers  &> /dev/null

In [ ]:
pip install PyAstronomy  &> /dev/null

# Lectura de datos

El archivo Postulantes.xlsx contiene los siguientes datos acerca de los postulantes a la UNALM:

- Edad, en años

- Puntaje RM, toma como máximo el valor 24

- Puntaje RV, toma como máximo el valor 24

- Puntaje Biología, toma como máximo el valor 6

- Puntaje Economía, toma como máximo el valor 4

- Tiempo, desde que terminó el colegio, en años

In [ ]:
import pandas as pd

In [ ]:
datos1 = pd.read_excel('04_Postulantes.xlsx', )
datos1

# Inspección gráfica

In [ ]:
datos1.hist(bins=10,figsize=(15,10))

Analizando la asimetría

In [ ]:
datos1['EDAD'].skew() # Para obtener el coeficiente de asimetría de una variable

In [ ]:
datos1.apply(lambda x: x.skew()).to_frame("Asimetría") # Para obtener el coeficiente de asimetría de todas las variables

Realizamos una prueba de hipótesis sobre el coeficiente de asimetría

$H0: as = 0$

$H1: as \neq 0$

Si $pvalor < \alpha$, se rechaza la hipótesis nula

In [ ]:
from scipy.stats import skewtest
datos1.apply(lambda x: skewtest(x, nan_policy='omit'))

A un nivel de significancia $\alpha=0.01$, se rechaza la hipótesis nula para las variables ..., mientras que ... Por lo tanto

In [ ]:
datos1.boxplot(figsize=(15,5))

In [ ]:
import matplotlib as plt
datos1.plot(kind='box', subplots=True, sharey=False, figsize=(15,5));

Verificando normalidad de los datos:

$H_0$: ...

$H_1$: ...

In [ ]:
from scipy.stats import shapiro
stat, p_valor = shapiro(datos1['EDAD'])
p_valor

In [ ]:
datos1.apply(lambda x: shapiro(x))

En conclusión, ...

In [ ]:
datos1.describe()

# **Filtros**

## Filtro z-score

In [ ]:
import numpy as np
from scipy.stats import zscore

In [ ]:
datos1_zscore = datos1.apply(zscore).abs()
datos1_zscore

### RM

In [ ]:
filtro_zscore_rm = datos1_zscore['PUNTAJE RM'] >3
filtro_zscore_rm

In [ ]:
np.where(filtro_zscore_rm)

In [ ]:
datos1[filtro_zscore_rm]

### RV

In [ ]:
filtro_score_rv = abs(datos1_zscore['PUNTAJE RV'])>3
np.where(filtro_score_rv)

In [ ]:
datos1[filtro_score_rv]

### EDAD

In [ ]:
filtro_score_edad = abs(datos1_zscore['EDAD'])>3
np.where(filtro_score_edad)

In [ ]:
datos1[filtro_score_edad]

## Chauvenet

In [ ]:
import numpy as np
from scipy.stats import norm

def chauvenet(data): # Creación de la función para filtro de Chauvenet
    mu            = data.mean()
    sigma         = data.std()
    distances     = abs(data - mu) / sigma
    probabilities = norm.sf(distances)
    outliers      = probabilities < 1 / (2 * len(data)) # regla de Chauvenet
    return outliers

In [ ]:
datos1_chauvenet = datos1.apply(chauvenet)
datos1_chauvenet

### RM

In [ ]:
filtro_chauvenet_rm = datos1_chauvenet['PUNTAJE RM']
np.where(filtro_chauvenet_rm)

In [ ]:
datos1[filtro_chauvenet_rm]

### RV

In [ ]:
filtro_chauvenet_rv = datos1_chauvenet['PUNTAJE RV']
np.where(filtro_chauvenet_rv)

### EDAD

In [ ]:
filtro_chauvenet_edad = datos1_chauvenet['EDAD']
np.where(filtro_chauvenet_edad)

In [ ]:
datos1[filtro_chauvenet_edad]

## Tukey

In [ ]:
def tukey(data):
    outliers  = []
    q1        = np.percentile(data, 25) # Cuartil 1 o percentil 25
    q3        = np.percentile(data, 75) # Cuartil 3 o percentil 75
    lwr_bound = q1-1.5*(q3-q1) # limite inferior # ric = q3-q1
    upr_bound = q3+1.5*(q3-q1) # limite superior # ric = q3-q1
    outliers  = np.array([(x < lwr_bound) or (x > upr_bound) for x in data]) # regla de Tukey
    return outliers

In [ ]:
datos1_tukey = datos1.apply(tukey)
datos1_tukey

### RM

In [ ]:
filtro_tukey_rm = datos1_tukey['PUNTAJE RM']
np.where(filtro_tukey_rm)

In [ ]:
datos1[filtro_tukey_rm]

### RV

In [ ]:
filtro_tukey_rv = datos1_tukey['PUNTAJE RV']
np.where(filtro_tukey_rv)

In [ ]:
datos1[filtro_tukey_rv]

### EDAD

In [ ]:
filtro_tukey_edad = datos1_tukey['EDAD']
np.where(filtro_tukey_edad)

In [ ]:
datos1[filtro_tukey_edad]

In [ ]:
np.intersect1d(np.where(filtro_tukey_rv), np.where(filtro_tukey_edad)) # este código solo vale para 2 variables

In [ ]:
datos1.loc[np.intersect1d(np.where(filtro_tukey_rv), np.where(filtro_tukey_edad))]

In [ ]:
conj1 = set(np.where(filtro_tukey_rm)[0])
conj2 = set(np.where(filtro_tukey_rv)[0])
conj3 = set(np.where(filtro_tukey_edad)[0])

conj1

In [ ]:
conj1.intersection(conj3)

In [ ]:
conj1.intersection(conj3,conj2)

# **Métodos inferenciales univariados**

## Test de Grubbs

In [ ]:
from outliers import smirnov_grubbs as grubbs

### RM

In [ ]:
datos1['PUNTAJE RM'].describe()

La media es 10.83, el máximo es 24 y el mínimo es -3.06

In [ ]:
10.83 - (-3.06), 24 - 10.83

La cantidad más alejada de la media es el mínimo, por lo tanto la hipótesis inicial analizará si el mínimo es un outlier.

$H_0:$ No existe un valor atípico inferior en los datos /// El mínimo no es un outlier


$H_1:$ Existe un valor atípico inferior en los datos /// El mínimo es un outlier

In [ ]:
len(grubbs.test(datos1['PUNTAJE RM'], alpha=.05))

En el código anterior, devuelve 1830 elementos, que corresponde a la cantidad de filas, por lo tanto no encontró ningún outlier inferior.


In [ ]:
len(grubbs.min_test(datos1['PUNTAJE RM'], alpha=.05))

In [ ]:
grubbs.min_test_indices(datos1['PUNTAJE RM'], alpha=.05)

In [ ]:
len(grubbs.max_test(datos1['PUNTAJE RM'], alpha=.05))

In [ ]:
grubbs.max_test_indices(datos1['PUNTAJE RM'], alpha=.05)

### RV

In [ ]:
len(grubbs.test(datos1['PUNTAJE RV'], alpha=.05))

### EDAD

In [ ]:
len(grubbs.test(datos1['EDAD'], alpha=.05))

In [ ]:
datos_sin_outliers = pd.DataFrame(grubbs.test(datos1['EDAD'], alpha=.05), columns=['EDAD'])
datos1[~datos1['EDAD'].isin(datos_sin_outliers['EDAD'])]

In [ ]:
len(grubbs.min_test(datos1['EDAD'], alpha=.05))

In [ ]:
len(grubbs.max_test(datos1['EDAD'], alpha=.05))

In [ ]:
grubbs.max_test_indices(datos1['EDAD'], alpha=.05)

## Test de Dixon

In [ ]:
import rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R -i datos1 # pasamos de Python a R

In [ ]:
%%R
install.packages("outliers")

In [ ]:
%%R
library(outliers)

Solo para mostrar el funcionamiento del código, se seleccionarán las 30 primeras filas.

### RM

In [ ]:
%%R
min(datos1$'PUNTAJE RM'[1:30]) - mean(datos1$'PUNTAJE RM'[1:30])

In [ ]:
%%R
max(datos1$'PUNTAJE RM'[1:30]) - mean(datos1$'PUNTAJE RM'[1:30])

In [ ]:
%%R
dixon.test(datos1$'PUNTAJE RM'[1:30])

A un nivel de significancia del $1\%$, el valor más alto no es un outlier.

In [ ]:
%%R
dixon.test(datos1$'PUNTAJE RM'[1:30], opposite = TRUE) # al colocar opposite TRUE, ya no analiza el máximo, sino ahora el mínimo.

A un nivel de significancia del $1\%$, el valor más bajo no es un outlier.

## Test de Rosner

In [ ]:
from PyAstronomy import pyasl

In [ ]:
# Ejemplo
pyasl.generalizedESD(np.array([10,1,3,2,3,4,3,4,6,300,400,15500,5,6,5,6,5,4,4,5,5,20]), 10, 0.05)

### RM

In [ ]:
pyasl.generalizedESD(np.array(datos1['PUNTAJE RM']), 10, 0.05)

### RV

In [ ]:
pyasl.generalizedESD(np.array(datos1['PUNTAJE RV']), 10, 0.05)

### EDAD

In [ ]:
pyasl.generalizedESD(np.array(datos1['EDAD']), 10, 0.05)

In [ ]:
datos1.loc[pyasl.generalizedESD(np.array(datos1['EDAD']), 10, 0.05)[1]]

# **Métodos multivariados de identificación de outliers**




## Identificación gráfica bivariada

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_matrix(datos1)
fig.update_layout(width=900, height=900)

## Distancia de Malahanobis

In [ ]:
# Aplicar .dropna().reset_index(drop=True) en caso haya perdidos, o imputar

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler() # definimos el estandarizador
datos1a = pd.DataFrame(scaler.fit_transform(datos1)) # aplicamos la estandarización, almacenando el resultado en un DataFrame
datos1a

In [ ]:
datos1a.columns = datos1.columns # las columnas del nuevo data frame datos1b serán las mismas que datos1
datos1a

In [ ]:
from numpy.linalg import pinv
from scipy.spatial.distance import mahalanobis

mean_vector    = np.mean(datos1a, axis = 0) ## datos1b.mean()
cov_matrix     = np.cov(datos1a.T)
inv_cov_matrix = np.linalg.inv(cov_matrix)

datos1a['Mahalanobis_Distance'] = datos1a.apply(lambda row: mahalanobis(row, mean_vector, inv_cov_matrix), axis=1)

In [ ]:
datos1a

In [ ]:
mean_mahalanobis = np.mean(datos1a['Mahalanobis_Distance'])
std_mahalanobis = np.std(datos1a['Mahalanobis_Distance'])
umbral = mean_mahalanobis + 3 * std_mahalanobis
umbral

In [ ]:
np.where(datos1a['Mahalanobis_Distance'] > umbral)

In [ ]:
datos1a[datos1a['Mahalanobis_Distance'] > umbral]

In [ ]:
outlier_mah = datos1[datos1a['Mahalanobis_Distance'] > umbral]
outlier_mah

In [ ]:
pd.DataFrame(np.mean(outlier_mah, axis=0)).T

In [ ]:
pd.DataFrame(np.mean(datos1, axis = 0)).T

## Elliptic Envelope

In [ ]:
round(np.mean(datos1a, axis = 0))

In [ ]:
from sklearn.covariance import EllipticEnvelope
ee = EllipticEnvelope(contamination = 0.02, support_fraction = 0.8, assume_centered=True).fit_predict(datos1a)
ee

In [ ]:
np.where(ee == -1)

In [ ]:
datos1.iloc[ee == -1]

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
isof = IsolationForest(contamination = 'auto').fit_predict(datos1) #detecta automáticamente el % de atípicos
np.where(isof == -1)

In [ ]:
datos1.iloc[isof == -1]

## Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(contamination = 'auto').fit_predict(datos1)
np.where(lof == -1)

In [ ]:
datos1.iloc[lof == -1]

In [ ]:
pos_atipicos1 = np.intersect1d(np.where(datos1a['Mahalanobis_Distance'] > umbral), np.where(ee == -1))
pos_atipicos1

In [ ]:
pos_atipicos2 = np.intersect1d(np.where(isof == -1), np.where(lof == -1))
pos_atipicos2

In [ ]:
conj_0 = set(np.where(datos1a['Mahalanobis_Distance'] > umbral)[0])
conj_1 = set(np.where(ee == -1)[0])
conj_2 = set(np.where(isof == -1)[0])
conj_3 = set(np.where(lof == -1)[0])
pos_atipicos3 = np.array(list(conj_0.intersection(conj_1,conj_2,conj_3)))
pos_atipicos3

In [ ]:
datos1.iloc[pos_atipicos3]

# Manipulación de outliers

In [ ]:
datos1.describe() # datos completos

In [ ]:
datos1.loc[pos_atipicos2].describe() # descripción de los datos atípicos

In [ ]:
datos1.drop(pos_atipicos2).describe() # descripción de los datos que no son atípicos

## Winsorizing

In [ ]:
from scipy.stats.mstats import winsorize

In [ ]:
datos1['PUNTAJE RM WINSO'] = winsorize(datos1['PUNTAJE RM'], limits=[0.01, 0.01]) # winsorize con percentiles 1 y 99
np.percentile(datos1['PUNTAJE RM'], [1,99])

In [ ]:
datos1[['PUNTAJE RM','PUNTAJE RM WINSO']].sort_values('PUNTAJE RM')

In [ ]:
datos1[['PUNTAJE RM','PUNTAJE RM WINSO']].describe()

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].hist(datos1['PUNTAJE RM'], bins=20)
axs[0].set_title('PUNTAJE RM')
axs[1].hist(datos1['PUNTAJE RM WINSO'], bins=20)
axs[1].set_title('PUNTAJE RM WINSO')

for ax in axs:
    ax.set_xlim((-5,25))
    ax.set_ylim((0,225))
plt.show()

In [ ]:
datos1['EDAD WINSO'] = winsorize(datos1['EDAD'], limits=[0.001, 0.01]) # winsorize con percentiles 0.1 y 99
np.percentile(datos1['EDAD'], [0.1,99])

In [ ]:
datos1[['EDAD','EDAD WINSO']].sort_values('EDAD')

In [ ]:
datos1[['EDAD','EDAD WINSO']].describe()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].hist(datos1['EDAD'], bins=20)
axs[0].set_title('EDAD')
axs[1].hist(datos1['EDAD WINSO'], bins=20)
axs[1].set_title('EDAD WINSO')

for ax in axs:
    ax.set_xlim((16,48))
    ax.set_ylim((0,1500))
plt.show()

## Trimming

Es útil cuando los valores atípicos son el resultado de errores de medición o entrada de datos incorrecta, o tienen un impacto desproporcionado en los resultados del análisis. Por ejemplo, la presencia de outliers altera en demasía los supuestos de la regresión.

También se puede usar en caso se trate de valores atípicos debido a la variabilidad natural de los datos, teniendo siempre en cuenta que la eliminación de datos puede reducir el tamaño de la muestra y, en algunos casos, eliminar información importante.

In [ ]:
# Calcular los valores de corte para la columna seleccionada
lower_cutoff, upper_cutoff = np.percentile(datos1['PUNTAJE RM'], [1,99])
lower_cutoff, upper_cutoff

In [ ]:
trim = (datos1['PUNTAJE RM'] >= lower_cutoff) & (datos1['PUNTAJE RM'] <= upper_cutoff) # si se encuentra entre los P1 y P99
trim.head(10)

In [ ]:
datos1.loc[trim] # solo muestra los registros para los que TAX se encuentra entre su percentil 1 y 99

In [ ]:
datos1.loc[trim].info()